In [1]:
#memasukan liblary yang dipakai 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
# Membaca dataset dari file CSV menggunakan Pandas
df = pd.read_csv("heart_cleveland_upload.csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


In [3]:
#mengubah data int ke fload / menyamakan seluruh data ke float 
df = df.astype({
    'age': float,
    'sex': float,
    'cp': float,
    'trestbps': float,
    'chol': float,
    'fbs': float,
    'restecg': float,
    'thalach': float,
    'exang': float,
    'oldpeak': float,
    'slope': float,
    'ca': float,
    'thal': float,
    'condition': float
})

In [4]:
df.dtypes #cek data tipe

age          float64
sex          float64
cp           float64
trestbps     float64
chol         float64
fbs          float64
restecg      float64
thalach      float64
exang        float64
oldpeak      float64
slope        float64
ca           float64
thal         float64
condition    float64
dtype: object

In [5]:
df.shape #cek jumlah kolom,baris

(297, 14)

In [6]:
# Memasukan data input dan output kedalam variable untuk training

X = df.drop("condition", axis=1).values  #input dari semua tabel dari df, kecuali coloum "condition"
y = df["condition"].values               #output hanya value dari coloum "condition"

# Memasukan data input dan output kedalam variable untuk testing
X_test = X
y_test = y 


In [7]:
# Menentukan perangkat yang akan digunakan (CPU atau CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Definisikan model
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)  #input 13
        self.fc2 = nn.Linear(128, 64)          #hidden layer 1 memiliki 128 neuron
        self.fc3 = nn.Linear(64, 32)           #hidden layer 2 memiliki 64 neuron
        self.fc4 = nn.Linear(32, 1)            #output 1
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x

# Membuat instance model
input_size = 13  # Sesuaikan dengan jumlah atribut
model = ANN(input_size).to(device)


In [8]:
# Definisikan loss function dan optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Konversi data ke Tensor dan pindahkan ke perangkat yang digunakan
X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
y_tensor = torch.tensor(y, dtype=torch.float32).unsqueeze(1).to(device)

# Melatih model
num_epochs = 297
batch_size = 64
num_samples = X_tensor.shape[0]
num_batches = num_samples // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = (batch + 1) * batch_size
        
        inputs = X_tensor[start_idx:end_idx]
        labels = y_tensor[start_idx:end_idx]
        
        # Melakukan forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Melakukan backward pass dan pembaruan bobot
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Mencetak loss pada setiap epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")
    

Epoch [1/297], Loss: 6.0742
Epoch [2/297], Loss: 0.6178
Epoch [3/297], Loss: 1.4695
Epoch [4/297], Loss: 0.4310
Epoch [5/297], Loss: 1.4822
Epoch [6/297], Loss: 0.4287
Epoch [7/297], Loss: 1.1421
Epoch [8/297], Loss: 0.4993
Epoch [9/297], Loss: 0.7132
Epoch [10/297], Loss: 0.6668
Epoch [11/297], Loss: 0.6005
Epoch [12/297], Loss: 0.7328
Epoch [13/297], Loss: 0.5907
Epoch [14/297], Loss: 0.7296
Epoch [15/297], Loss: 0.6044
Epoch [16/297], Loss: 0.7162
Epoch [17/297], Loss: 0.6240
Epoch [18/297], Loss: 0.7045
Epoch [19/297], Loss: 0.6422
Epoch [20/297], Loss: 0.6911
Epoch [21/297], Loss: 0.6532
Epoch [22/297], Loss: 0.6752
Epoch [23/297], Loss: 0.6560
Epoch [24/297], Loss: 0.6611
Epoch [25/297], Loss: 0.6552
Epoch [26/297], Loss: 0.6517
Epoch [27/297], Loss: 0.6535
Epoch [28/297], Loss: 0.6462
Epoch [29/297], Loss: 0.6506
Epoch [30/297], Loss: 0.6424
Epoch [31/297], Loss: 0.6459
Epoch [32/297], Loss: 0.6387
Epoch [33/297], Loss: 0.6401
Epoch [34/297], Loss: 0.6348
Epoch [35/297], Loss: 0

In [9]:

# Mengkonversi data pengujian ke Tensor dan memindahkannya ke perangkat yang digunakan
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1).to(device)

# Menjadikan model ke mode evaluasi (tidak ada pembaruan bobot)
model.eval()

# Melakukan prediksi pada data pengujian
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted_labels = (outputs >= 0.5).float()  # Menggunakan threshold 0.5 untuk klasifikasi biner

# Menghitung metrik evaluasi
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, predicted_labels.cpu())
precision = precision_score(y_test, predicted_labels.cpu())
recall = recall_score(y_test, predicted_labels.cpu())
f1 = f1_score(y_test, predicted_labels.cpu())

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

Accuracy: 0.8552
Precision: 0.8917
Recall: 0.7810
F1-score: 0.8327


In [10]:
#menyimpan hasil training
torch.save(model.state_dict(), 'asep1.pth')